In [4]:
import pandas as pd
import numpy as np
import itertools
import pickle
import os

# Gather

In [3]:
decks = pd.read_csv('C:/Users/muroc/Documents/MTG/data/decks.csv')

FileNotFoundError: [Errno 2] File b'C:/Users/muroc/Documents/MTG/data/decks.csv' does not exist: b'C:/Users/muroc/Documents/MTG/data/decks.csv'

In [9]:
decks = pd.read_csv('C:\\Users\\muroc\\OneDrive\\Documents\\programs\\python\\MTGA_rarity_analysis\\data\\decks.csv')

In [10]:
decks_clean = decks.copy()

In [5]:
players = np.load('C:/Users/muroc/Documents/MTG/data/players.npy')

ValueError: Cannot load file containing pickled data when allow_pickle=False

# Assess

In [11]:
decks_clean.head()

,deck_list,player,url,name
0,"2 Forest, 1 Island, 1 Mountain, 2 Swamp, 3 Bre...",Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control
1,"4 Shock, 1 Forest, 1 Island, 1 Mountain, 1 Swa...",Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals
2,"3 Forest, 1 Island, 2 Plains, 1 Swamp, 4 Breed...",Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value
3,"1 Island, 1 Mountain, 2 Plains, 3 Swamp, 2 Blo...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_
4,"1 Island, 1 Mountain, 1 Plains, 4 Hallowed Fou...",Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW


In [5]:
decks_clean.tail()

,deck_list,player,url,name
488,"4 Planar Cleansing, 7 Island, 5 Plains, 4 Hall...",Yokokawa_Yuuta,https://www.mtgtop8.com/event?e=23907&d=366416...,UW_Control
489,"1 Disenchant, 7 Island, 4 Plains, 4 Hallowed F...",zer1997,https://www.mtgtop8.com/event?e=23772&d=365157...,UW_Control
490,"17 Plains, 4 Loyal Pegasus, 4 Faerie Guidemoth...",Brandon_Krussow,https://www.mtgtop8.com/event?e=23943&d=366839...,White_Weenie
491,"2 Disenchant, 17 Plains, 4 Loyal Pegasus, 4 Fa...",Nematov_Ruslan,https://www.mtgtop8.com/event?e=23578&d=363291...,White_Weenie
492,NaN,Aki_Vainio,https://www.mtgtop8.com/event?e=23996&d=367291...,Rakdos_Knights


In [6]:
decks_clean[decks_clean.astype(str)['deck_list'] == "['nan']"]

,deck_list,player,url,name


In [7]:
decks_clean.deck_list.iloc[492]

nan

In [4]:
np.count_nonzero(deck_list)

0

## Assessment
- [done] deck_list column has , for delimiter but names can also have a comma
- cards are in one column
- card number and card name are in the same column
- [done] null values in `deck_list` column
- rarities not present in the data

# Clean

## Define
- replace ',' with '|' in `deck_list` column

## Code

In [58]:
decks_clean.deck_list = decks_clean.deck_list.str.replace(', (?=\d)', '|')

In [9]:
decks_clean.to_csv('C:/Users/muroc/Documents/MTG/data/decks_clean.csv', index=False)

## Test

In [59]:
decks_clean.head()

,deck_list,player,url,name
0,2 Forest|1 Island|1 Mountain|2 Swamp|3 Breedin...,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control
1,4 Shock|1 Forest|1 Island|1 Mountain|1 Swamp|1...,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals
2,3 Forest|1 Island|2 Plains|1 Swamp|4 Breeding ...,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value
3,1 Island|1 Mountain|2 Plains|3 Swamp|2 Blood C...,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_
4,1 Island|1 Mountain|2 Plains|3 Swamp|2 Blood C...,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM


## Define
- make string objects in `deck_list` into a list

## Code

In [11]:
def convert(string):
    li = list(str(string).split('|'))
    return li

In [12]:
decks = []
for i in np.arange(decks_clean.shape[0]):
    l = convert(decks_clean.deck_list[i])
    decks.append(l)

decks_clean.deck_list = decks

## Test

In [13]:
decks_clean.head()

,deck_list,player,url,name
0,"[2 Forest, 1 Island, 1 Mountain, 2 Swamp, 3 Br...",Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control
1,"[4 Shock, 1 Forest, 1 Island, 1 Mountain, 1 Sw...",Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals
2,"[3 Forest, 1 Island, 2 Plains, 1 Swamp, 4 Bree...",Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value
3,"[1 Island, 1 Mountain, 2 Plains, 3 Swamp, 2 Bl...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_
4,"[1 Island, 1 Mountain, 2 Plains, 3 Swamp, 2 Bl...",Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23442&d=361922...,Mardu_DOOM


## Define
- filter for any lists in `deck_list` that have null values in it

## Code

In [14]:
decks_clean.deck_list = decks_clean[decks_clean.astype(str)['deck_list'] != "['nan']"]

## Test

In [15]:
decks_clean[decks_clean.astype(str)['deck_list'] == "['nan']"]

,deck_list,player,url,name


In [16]:
decks_clean.to_csv('C:/Users/muroc/Documents/MTG/data/decks_clean.csv')

## Define
- make `deck_list` into a numpy array

## Code

In [13]:
deck_list = decks_clean.deck_list.to_list()

## Test

In [14]:
type(deck_list)

list

In [15]:
deck_list[0]

"2 Forest, 1 Island, 1 Mountain, 2 Swamp, 3 Breeding Pool, 2 Mystical Dispute, 2 Cauldron Familiar, 2 Murderous Rider, 4 Gilded Goose, 4 Once Upon a Time, 2 Trail of Crumbs, 4 Wicked Wolf, 4 Oko, Thief of Crowns, 3 Witch's Oven, 4 Fabled Passage, 2 Korvold, Fae-Cursed King, 3 Stomping Ground, 2 Temple of Malady, 4 Aether Gust, 4 Noxious Grasp, 4 Overgrown Tomb, 2 Watery Grave, 3 Thrashing Brontodon, 3 Duress, 1 Ugin, the Ineffable, 2 Paradise Druid, 4 Mayhem Devil, 1 Nicol Bolas, Dragon-God, "

In [16]:
np.count_nonzero(deck_list)

451